# Úkol č. 1 - předzpracování dat a binární klasifikace

  * **Deadline je do 3. 11. 2022, 23:59:59**, pokud odevzdáte úkol do 10. 11. 2022, 23:59:59, budete penalizování -4 body, pozdější odevzdání je bez bodu.
  * V rámci tohoto úkolu se musíte vypořádat s příznaky, které jsou různých typů.
  * Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **12 bodů**:
  * Využívejte buňky typu `Markdown` k vysvětlování Vašeho postupu. Za nepřehlednost budeme strhávat body.
  * V notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na podmnožiny potřebné k trénování a evaluaci modelu (optimálně tedy trénovací, validační a testovací).
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit. Pro průzkum dat využívejte vizualizace.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor též odevzdejte (uložte do projektu vedle notebooku).
  * Ukázka prvních řádků souboru *results.csv*:
  
```
ID,survived
1000,0
1001,1
...
```

**Další body zadání** za případné další body (můžete si vybrat, maximum bodů za úkol je každopádně 16 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte na testovací množině. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zaměřte se na optimální předzpracování dat. Zabývejte se tím, jak nejlépe zpracovat a reprezentovat kategoriální příznaky. Také zkuste data normalizovat. Zaměřte se na vliv těchto kroků na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-ML1/homeworks/index.html.
  * Vytvořte i csv soubor s predikcemi a uložte ho v rámci projektu (`results.csv`, vedle ipython notebooku).
  * Opravující Vám ve výjimečných případech může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale stěžejní a má hlavní vliv na hodnocení.

Imports

In [1]:
import pandas as pd
import numpy as np

from statistics import mean
from math import floor

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ParameterGrid
import sklearn.metrics as metrics

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
%matplotlib inline 

Load data and check for Null values

In [92]:
df = pd.read_csv("data.csv")
# df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 53 to 652
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ID         1000 non-null   int64   
 1   survived   1000 non-null   int64   
 2   pclass     1000 non-null   int64   
 3   name       1000 non-null   object  
 4   sex        1000 non-null   object  
 5   age        1000 non-null   float64 
 6   sibsp      1000 non-null   int64   
 7   parch      1000 non-null   int64   
 8   ticket     1000 non-null   object  
 9   fare       999 non-null    float64 
 10  cabin      222 non-null    object  
 11  embarked   999 non-null    object  
 12  home.dest  573 non-null    object  
 13  Interval   1000 non-null   category
dtypes: category(1), float64(2), int64(5), object(6)
memory usage: 110.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 53 to 652
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ID         1000 non-null   int64   
 1   survived   1000 non-null   int64   
 2   pclass     1000 non-null   int64   
 3   name       1000 non-null   object  
 4   sex        1000 non-null   object  
 5   age        1000 non-null   float64 
 6   sibsp      1000 non-null   int64   
 7   parch      1000 non-null   int64   
 8   ticket     1000 non-null   object  
 9   fare       999 non-null    float64 
 10  cabin      222 non-null    object  
 11  embarked   999 non-null    object  
 12  home.dest  573 non-null    object  
 13  Interval   1000 non-null   category
dtypes: category(1), float64(2), int64(5), object(6)
memory usage: 110.5+ KB


None

Convert the embarked column into 0/1/2.

In [3]:
df.embarked = df.embarked.replace({"C": 0, "Q": 1, "S": 2})

Missing values in columns I plan to use are filled in with the mean of that column.

In [4]:
df.age = df.age.fillna(df.age.mean())
df.fare = df.fare.fillna(df.fare.mean())
df.embarked = df.embarked.fillna(floor(df.embarked.mean()))
df.isnull().sum()

ID             0
survived       0
pclass         0
name           0
sex            0
age            0
sibsp          0
parch          0
ticket         0
fare           0
cabin        778
embarked       0
home.dest    427
dtype: int64

There is not enough data for cabin and home destination, so I drop it.

In [5]:
df = df.drop(columns=["cabin", "home.dest"])

Convert sex into 0/1.

In [6]:
df.sex = df.sex.replace({"male": 0, "female": 1})

Checking what has what impact on surviving.

In [7]:
cor_matrix = df.corr(numeric_only=True)
cor_matrix

,ID,survived,pclass,sex,age,sibsp,parch,fare,embarked
ID,1.000000,-0.030423,0.038144,-0.019001,-0.019708,0.000396,-0.032359,-0.022320,0.025025
survived,-0.030423,1.000000,-0.306425,0.533893,-0.035877,-0.022323,0.086493,0.241509,-0.163003
pclass,0.038144,-0.306425,1.000000,-0.121329,-0.353733,0.059313,0.026576,-0.545761,0.178336
sex,-0.019001,0.533893,-0.121329,1.000000,-0.056748,0.123758,0.244412,0.181520,-0.094325
age,-0.019708,-0.035877,-0.353733,-0.056748,1.000000,-0.202125,-0.126964,0.169821,-0.074440
sibsp,0.000396,-0.022323,0.059313,0.123758,-0.202125,1.000000,0.370997,0.140273,0.066780
parch,-0.032359,0.086493,0.026576,0.244412,-0.126964,0.370997,1.000000,0.199704,0.062697
fare,-0.022320,0.241509,-0.545761,0.181520,0.169821,0.140273,0.199704,1.000000,-0.230909
embarked,0.025025,-0.163003,0.178336,-0.094325,-0.074440,0.066780,0.062697,-0.230909,1.000000


Sibling, spouses, parents and children all have small impact, so I combined them into having **family**.

In [8]:
def family(x):
    return 1 if (x["sibsp"] + x["parch"] > 0) else 0

df["family"] = df.apply(family, axis=1)
df = df.drop(columns=["sibsp", "parch"])

I could use titles from the name column, but it seems like too little reward for too much effort and I would need to convert it from string somehow. Same goes for the ticket.

In [9]:
df = df.drop(columns=["name", "ticket"])

Now I have only numeric values.

In [10]:
display(df.dtypes)

ID            int64
survived      int64
pclass        int64
sex           int64
age         float64
fare        float64
embarked    float64
family        int64
dtype: object

Last look at the data.

In [11]:
df


,ID,survived,pclass,sex,age,fare,embarked,family
0,0,1,3,0,27.000000,7.7958,2.0,0
1,1,1,3,0,45.000000,8.0500,2.0,0
2,2,0,2,0,30.006692,0.0000,2.0,0
3,3,1,3,1,27.000000,11.1333,2.0,1
4,4,0,1,0,28.000000,82.1708,0.0,1
...,...,...,...,...,...,...,...,...
995,995,1,1,1,22.000000,151.5500,2.0,0
996,996,0,2,0,21.000000,73.5000,2.0,0
997,997,1,1,1,24.000000,69.3000,0.0,0
998,998,0,3,0,30.006692,69.5500,2.0,1


Separate the target variable from the rest of the data

In [12]:
ydata = df.survived
Xdata = df.drop("survived", axis = 1)

Set random seed and split data into train, test, validattion sets.

In [13]:
rd_seed = 666
Xtrain, Xtest, ytrain, ytest = train_test_split(Xdata, ydata, test_size=0.5, random_state=rd_seed) 

In [14]:
Xval, Xtest, yval, ytest = train_test_split(Xtest, ytest, test_size=0.5, random_state=rd_seed) 
print(f"Train dimension, X: {Xtrain.shape}, y: {ytrain.shape}")
print(f"Val dimension, X: {Xval.shape}, y: {yval.shape}")
print(f"Test dimension, X: {Xtest.shape}, y: {ytest.shape}")

Train dimension, X: (500, 7), y: (500,)
Val dimension, X: (250, 7), y: (250,)
Test dimension, X: (250, 7), y: (250,)


Parameter grid for all possible combinations.

In [15]:
param_grid = {
    'max_depth': range(1,42), 
    'criterion': ['entropy', 'gini', 'log_loss']
}
param_comb = ParameterGrid(param_grid)
val_acc = []

Loop through the combinations and find the best one.

In [16]:
for params in param_comb:
    dt = DecisionTreeClassifier(max_depth=params['max_depth'], criterion=params['criterion'])
    dt.fit(Xtrain, ytrain)
    val_acc.append(metrics.accuracy_score(yval, dt.predict(Xval)))

Print out the best parameters and accuracy score for those parameters.

In [17]:
best_params = param_comb[np.argmax(val_acc)]
print(f'best params {best_params}')

dt = DecisionTreeClassifier(max_depth=best_params['max_depth'], criterion=best_params['criterion'])
dt.fit(Xtrain, ytrain)

print(f'accuracy score (train): {dt.score(Xtrain, ytrain):0.4f}')
print(f'accuracy score (validation): {dt.score(Xval, yval):0.4f}')
print(f'accuracy score (test): {metrics.accuracy_score(ytest, dt.predict(Xtest)):0.4f}')

best params {'max_depth': 3, 'criterion': 'entropy'}
accuracy score (train): 0.8320
accuracy score (validation): 0.7800
accuracy score (test): 0.8040


I tried the same for KNN, but the results were worse.

In [18]:
val_acc = []

kneighbors = range(3,15)
for k in kneighbors:
    clf = KNeighborsClassifier(n_neighbors = k)
    clf.fit(Xtrain, ytrain)
    val_acc.append(clf.score(Xval, yval))

best_k = np.argmax(val_acc)

clf = KNeighborsClassifier(n_neighbors = best_k)
clf.fit(Xtrain, ytrain)

print(f'Best number of neighbors {best_k}')
print(f'accuracy score (train): {clf.score(Xtrain, ytrain):0.4f}')
print(f'accuracy score (validation): {clf.score(Xval, yval):0.4f}')
print(f'accuracy score (test): {metrics.accuracy_score(ytest, clf.predict(Xtest)):0.4f}')

Best number of neighbors 4
accuracy score (train): 0.7360
accuracy score (validation): 0.5720
accuracy score (test): 0.6040


So I tried to scale the data.

In [19]:
scaler = MinMaxScaler()
Xtrain_scaled = scaler.fit_transform(Xtrain)
Xval_scaled = scaler.transform(Xval)

It's better, but still worse.

In [20]:
val_acc = []
kneighbors = range(3,15)

for k in kneighbors:
    clf = KNeighborsClassifier(n_neighbors = k)
    clf.fit(Xtrain_scaled, ytrain)
    val_acc.append(clf.score(Xval_scaled, yval))

best_k = np.argmax(val_acc)

clf = KNeighborsClassifier(n_neighbors = best_k)
clf.fit(Xtrain_scaled, ytrain)

print(f'Best number of neighbors {best_k}')
print(f'accuracy score (train): {clf.score(Xtrain_scaled, ytrain):0.4f}')
print(f'accuracy score (validation): {clf.score(Xval_scaled, yval):0.4f}')
print(f'accuracy score (test): {metrics.accuracy_score(ytest, clf.predict(Xtest)):0.4f}')

Best number of neighbors 4
accuracy score (train): 0.8440
accuracy score (validation): 0.7400
accuracy score (test): 0.6760


C:\Users\Brumda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


With my model, the decision tree is not really good, but still better than the KNN.

Load the evaluation data and prepare them the same way as the original ones.

In [21]:
eval = pd.read_csv("evaluation.csv")

eval.embarked = eval.embarked.replace({"C": 0, "Q": 1, "S": 2})
eval.age = eval.age.fillna(eval.age.mean())
eval.fare = eval.fare.fillna(eval.fare.mean())
eval.embarked = eval.embarked.fillna(floor(eval.embarked.mean()))
eval = eval.drop(columns=["cabin", "home.dest"])
eval.sex = eval.sex.replace({"male": 0, "female": 1})
def family(x):
    return 1 if (x["sibsp"] + x["parch"] > 0) else 0

eval["family"] = eval.apply(family, axis=1)
eval = eval.drop(columns=["sibsp", "parch"])
eval = eval.drop(columns=["name", "ticket"])
eval.isnull().sum()
# eval

ID          0
pclass      0
sex         0
age         0
fare        0
embarked    0
family      0
dtype: int64

Trained decision tree is predicting the survive value for new data.

In [22]:
prediction = dt.predict(eval)
print(prediction.shape)

(309,)


Putting everything into result.csv file.

In [23]:
ID = eval['ID']
test_Survived = pd.Series(dt.predict(eval), name="Survived")

results = pd.concat([ID, test_Survived], axis=1)

results.to_csv("result.csv", index=False)